## Separacion de dataframes
### Separamos los dataframes en train, validation y test

In [350]:
import sys
import Utilidades as ut
import pandas as pd
import numpy as np

PREPARAR_REDES_NEURONALES = True

df = pd.read_pickle("Archivos/Entrenamiento_limpio.pkl")
df_test = pd.read_pickle("Archivos/Test_limpio.pkl")

#Pasamos la columna Stage a entero.
df['Stage'] = (df['Stage'] == 'Closed Won').astype('int64')

#Convertimos Total_Amount y Total_Taxable_Amount a escala logaritmica
#Sumo uno para evitar el cero

df['Total_Amount(USD)'] = np.log10(df['Total_Amount(USD)'] + 1)
df['Total_Taxable_Amount(USD)'] = np.log10(df['Total_Taxable_Amount(USD)'] + 1)
df_test['Total_Amount(USD)'] = np.log10(df_test['Total_Amount(USD)'] + 1)
df_test['Total_Taxable_Amount(USD)'] = np.log10(df_test['Total_Taxable_Amount(USD)'] + 1)

#Realizamos un split del df original a df_train, df_validation

split_date = np.datetime64('2018-01-01')

df = df.sort_values('Opportunity_Created_Date', ascending=True)
df_test = df_test.sort_values('Opportunity_Created_Date', ascending=True)

df_train = df.loc[df.Opportunity_Created_Date < split_date]
df_validation = df.loc[df.Opportunity_Created_Date >= split_date]
train_size = df_train.shape[0]
validation_size = df_validation.shape[0]

split_proportion = validation_size/(validation_size + train_size)

print(f'Se realiza un split {round(1-split_proportion, 2)}-{round(split_proportion, 2)} entre train-validation\n')
df.loc[:, 'Opportunity_Total_Amount'] = df.groupby('Opportunity_ID')['Total_Amount(USD)'].transform('sum')

df.head(3).T

Se realiza un split 0.73-0.27 entre train-validation



/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,16104,16105,16106
Region,EMEA,APAC,Americas
Territory,France,Philippines,NW America
"Pricing, Delivery_Terms_Quote_Appr",0,1,1
"Pricing, Delivery_Terms_Approved",0,1,0
Bureaucratic_Code_0_Approval,0,1,1
Bureaucratic_Code_0_Approved,0,1,0
Submitted_for_Approval,0,0,0
Bureaucratic_Code,Bureaucratic_Code_4,Bureaucratic_Code_3,Bureaucratic_Code_5
Account_Created_Date,2013-07-27 00:00:00,2014-01-22 00:00:00,2013-08-22 00:00:00
Billing_Country,France,Germany,United States


## Feature Engineering
### Preparamos los datos para los modelos de machine learning

### Posibles features

- Opportunity_Total_Amount: El total amount de todos los productos de la oportunidad.
- Opportunity_Size: La cantidad de productos vendidos en la oportunidad.
- Planned_Opportunity_Duration: El tiempo estimado de duracion de la oportunidad.
- Actual_Opportunity_Duration: El tiempo entre que se crea la oportunidad hasta que es modificada por ultima vez.
- Opportunity_Taxable_Rate: El ratio de Total_Taxable_Amount / Opportunity_Total_Amount
- Opportunity_Currency: Moneda en la cual esta expresado el total amount de la oportunidad. (Estaba en el df original)
- Product_Amount_Deviation_of_Product_Family_rate: Desviacion de la media (o mediana) del precio del producto segun la familia del producto.
- Month: (Cambiar de formato año-mes a solo mes)
- Year: (Formada a partir de la columna original Month)
- Year-Month: Formato año-mes (formada a partir de la columna original Month)
- Avg_Product_Duration: Vida media del producto por region.
- Opportunity_TRF: TRF de la suma de productos de la oportunidad.


### Mezcla de features

- Opportunity_Duration_Ratio: Ratio entre duracion real de la oportunidad y duracion estimada Actual_Opportunity_Duration / Actual_Opportunity_Duration.
- Opportunity_Total_Amount_Region_avg: Total amount promedio por region.
- Opportunity_Total_Amount_Region_std: Desviacion estandar del total amount por region.
- Opportunity_Total_Amount_Region_avg_Ratio: Opportunity_Total_Amount / Opportunity_Total_Amount_Region_avg
- Opportunity_Total_Amount_Region_std_Ratio: Opportunity_Total_Amount / Opportunity_Total_Amount_Region_std
- Opportunity_TRF_Region_avg: Media de TRF de oportunidad para la region.
- Opportunity_TRF_Region_std: Desviacion estandar de TRF de oportunidad para la region.
- Opportunity_TRF_Region_avg_Ratio: Opportunity_TRF / Opportunity_TRF_Region_avg
- Opportunity_TRF_Region_std_Ratio: Opportunity_TRF / Opportunity_TRF_Region_std

### Features en el tiempo

- Product_Family_Total_Sells_Region_this_week: Cantidad de ventas de un producto de product Family en los ultimos 7 dias, para una Region.
- Product_Family_Total_Sells_Region_last_week: Cantidad de ventas de un producto de product Family en la semana pasada, para una Region.
- Product_Family_Total_Sells_Region_last_week_ratio: Product_Family_Total_Sells_Region_this_week / Product_Family_Total_Sells_Region_last_week

- Product_Family_Total_Sells_Region_this_month: Cantidad de ventas de un producto de product Family en el ultimo mes, para una Region.
- Product_Family_Total_Sells_Region_last_month: Cantidad de ventas de un producto de product Family en el mes pasado, para una Region.
- Product_Family_Total_Sells_Region_last_month_ratio: Product_Family_Total_Sells_Region_this_month / Product_Family_Total_Sells_Region_last_month

- Product_Family_Total_Sells_Region_this_quarter: Cantidad de ventas de un producto de product Family en el ultimo trimestre, para una Region.
- Product_Family_Total_Sells_Region_last_quarter: Cantidad de ventas de un producto de product Family en el trimestre pasado, para una Region.
- Product_Family_Total_Sells_Region_last_quarter_ratio: Product_Family_Total_Sells_Region_this_quarter / Product_Family_Total_Sells_Region_last_quarter




### Armado de los features

In [351]:
# Armamos una funcion para aplicar el mismo procedimiento a los distintos dataframes

def feature_creation(df):

    ##### Opportunity_Total_Amount
    
    df['Opportunity_Total_Amount'] = df.groupby('Opportunity_ID')['Total_Amount(USD)'].transform('sum')
    
    ### Opportunity_Size
    
    df['Opportunity_Size'] = df.groupby('Opportunity_ID')['Total_Amount(USD)'].transform('count')
    
    ### Planned_Deliver_Duration
    
    df['Planned_Deliver_Duration'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']) / np.timedelta64(1, 'D')
    
    ### Actual_Opportunity_Duration
    
    df['Actual_Opportunity_Duration'] = (df['Last_Modified_Date'] - df['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
    
    ### OBS: Las dos siguientes tienen valores negativos, no tiene ningun sentido. Ver que hacer.
    
    ### Planned_Time_Until_Deliver
    
    df['Planned_Time_Until_Deliver'] = (df['Planned_Delivery_Start_Date'] - df['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
    
    ### Planned_Opportunity_Duration
    
    df['Planned_Opportunity_Duration'] = (df['Planned_Delivery_End_Date'] - df['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
    
    ### Opportunity_Taxable_Rate
    
    df['Opportunity_Taxable_Rate'] = df['Total_Taxable_Amount(USD)'] / df['Opportunity_Total_Amount']
    
    ### Opportunity_Currency
    
    ## YA INCLUIDA EN ASP_CURRENCY, VER SI VALE LA PENA CAMBIARLE EL NOMBRE A LA COLUMNA
    
    ### Product_Amount_Deviation_of_Product_Family_rate
    
    product_family_median = df.groupby('Product_Family')['Total_Amount(USD)'].transform('median')
    df['Product_Amount_Deviation_of_Product_Family_rate'] = (df['Total_Amount(USD)'] - product_family_median) / product_family_median
    
    ### Year-Month
    
    df = df.astype({'Month' : 'datetime64'})
    
    df['Year-Month'] = df['Month']
    
    #df = df.astype({'Year-Month' : 'category'})
    
    ### Month
    
    #df = df.astype({'Month' : object})
    #
    #for (idx, v) in zip(df['Year-Month'].index, df['Year-Month'].values):
    #    anio = v[:4]
    #    mes = v[7:]
    #    df.loc[idx, 'Month'] = mes
    #
    
    
    df['Month'] = df['Month'].dt.month
    
    ### Avg_Product_Duration
    
    #Ver como vincular la vida media de los productos.
    
    ### Opportunity_TRF
    
    df['Opportunity_TRF'] = df.groupby('Opportunity_ID')['TRF'].transform('sum')
    
    return df

### Mezcla de features

In [352]:
# Armamos una funcion para aplicar el mismo procedimiento a los distintos dataframes

def feature_composition(df):

    ### Opportunity_Duration_Ratio
    
    df['Opportunity_Duration_Ratio'] = df['Planned_Opportunity_Duration'] / df['Actual_Opportunity_Duration']
    
    #### CHEQUEAR ESTO #####################
    # Llenamos los posibles inf que hayan surgido con un numero "muy grande".
    
    df['Opportunity_Duration_Ratio'].replace(to_replace=np.inf, value=10000, inplace=True)
    df['Opportunity_Duration_Ratio'].replace(to_replace=-np.inf, value=-10000, inplace=True)
    
    #########################################
    
    ### Opportunity_Total_Amount_Region_avg
    
    df['Opportunity_Total_Amount_Region_avg'] = df.groupby('Region')['Opportunity_Total_Amount'].transform('mean')
    
    ### Opportunity_Total_Amount_Region_std
    
    df['Opportunity_Total_Amount_Region_std'] = df.groupby('Region')['Opportunity_Total_Amount'].transform('std')
    
    ### Opportunity_Total_Amount_Region_avg_Ratio
    
    df['Opportunity_Total_Amount_Region_avg_Ratio'] = df['Opportunity_Total_Amount'] / df['Opportunity_Total_Amount_Region_avg']
    
    ### Opportunity_Total_Amount_Region_std_Ratio
    
    df['Opportunity_Total_Amount_Region_std_Ratio'] = df['Opportunity_Total_Amount'] / df['Opportunity_Total_Amount_Region_std']
    
    ### Opportunity_TRF_Region_avg
    
    df['Opportunity_TRF_Region_avg'] = df.groupby('Region')['TRF'].transform('mean')
    
    ### Opportunity_TRF_Region_std
    
    df['Opportunity_TRF_Region_std'] = df.groupby('Region')['TRF'].transform('std')
    
    ### Opportunity_Total_Amount_Region_avg_Ratio
    
    df['Opportunity_TRF_Region_avg_Ratio'] = df['Opportunity_TRF'] / df['Opportunity_TRF_Region_avg']
    
    ### Opportunity_Total_Amount_Region_std_Ratio
    
    df['Opportunity_TRF_Region_std_Ratio'] = df['Opportunity_TRF'] / df['Opportunity_TRF_Region_std']
    
    ### Opportunity_Duration_by_Billing_Country
    
    df["Opportunity_Duration_by_Billing_Country"] = df.groupby("Billing_Country")["Actual_Opportunity_Duration"].transform("mean")
    
    ### Opportunity_Duration_by_Account_Type
    
    df["Opportunity_Duration_by_Account_Type"] = df.groupby("Account_Type")["Actual_Opportunity_Duration"].transform("mean")
    
    ## ASP_by_Region_mean
    
    df["ASP_by_Region_mean"] = df.groupby("Region")["ASP_(converted)"].transform("mean")
    
    ## ASP_by_Region_std
    
    df["ASP_by_Region_std"] = df.groupby("Region")["ASP_(converted)"].transform("std")
    
    ## ASP_by_Billing_Country_mean
    
    df["ASP_by_Billing_Country_mean"] = df.groupby("Billing_Country")["ASP_(converted)"].transform("mean")
    
    ## ASP_by_Billing_Country_std
    
    df["ASP_by_Billing_Country_std"] = df.groupby("Billing_Country")["ASP_(converted)"].transform("std")
    
    ## Total_Amount_by_Billing_Country_mean
    
    df["Total_Amount_by_Billing_Country_mean"] = df.groupby("Billing_Country")["Total_Amount(USD)"].transform("mean")
    
    ## Total_Amount_by_Billing_Country_std
    
    df["Total_Amount_by_Billing_Country_std"] = df.groupby("Billing_Country")["Total_Amount(USD)"].transform("std")
    
    ## Buro_Approved_by_Billing_Country_mean
    
    df["Buro_Approved_by_Billing_Country_mean"] = df.groupby("Billing_Country")["Bureaucratic_Code_0_Approved"].transform("mean")
    
    ## Buro_Approved_by_Billing_Country_std
    
    df["Buro_Approved_by_Billing_Country_std"] = df.groupby("Billing_Country")["Bureaucratic_Code_0_Approved"].transform("std")
    
    ## Opportunity_Duration_by_Product_Family_mean
    
    df["Opportunity_Duration_by_Product_Family_mean"] = df.groupby("Product_Family")["Actual_Opportunity_Duration"].transform("mean")
    
    ## Opportunity_Duration_by_Product_Family_std
    
    df["Opportunity_Duration_by_Product_Family_std"] = df.groupby("Product_Family")["Actual_Opportunity_Duration"].transform("std")
    
    ## Total_Amount_by_Product_Family_mean
    
    df["Total_Amount_by_Product_Family_mean"] = df.groupby("Product_Family")["Total_Amount"].transform("mean")
    
    ## Total_Amount_by_Product_Family_std
    
    df["Total_Amount_by_Product_Family_std"] = df.groupby("Product_Family")["Total_Amount"].transform("std")
    
    ## Buro_Approved_by_Product_Family_mean
    
    df["Buro_Approved_by_Product_Family"] = df.groupby("Product_Family")["Bureaucratic_Code_0_Approved"].transform("mean")
    
    ## Buro_Approved_by_Product_Family_std
    
    df["Buro_Approved_by_Product_Family"] = df.groupby("Product_Family")["Bureaucratic_Code_0_Approved"].transform("std")
    
    return df

## Features en el tiempo

- Product_Family_Total_Sells_Region_this_week: Cantidad de ventas de un producto de product Family en los ultimos 7 dias, para una Region.
- Product_Family_Total_Sells_Region_last_week: Cantidad de ventas de un producto de product Family en la semana pasada, para una Region.
- Product_Family_Total_Sells_Region_last_week_ratio: Product_Family_Total_Sells_Region_this_week / Product_Family_Total_Sells_Region_last_week

- Product_Family_Total_Sells_Region_this_month: Cantidad de ventas de un producto de product Family en el ultimo mes, para una Region.
- Product_Family_Total_Sells_Region_last_month: Cantidad de ventas de un producto de product Family en el mes pasado, para una Region.
- Product_Family_Total_Sells_Region_last_month_ratio: Product_Family_Total_Sells_Region_this_month / Product_Family_Total_Sells_Region_last_month

- Product_Family_Total_Sells_Region_this_quarter: Cantidad de ventas de un producto de product Family en el ultimo trimestre, para una Region.
- Product_Family_Total_Sells_Region_last_quarter: Cantidad de ventas de un producto de product Family en el trimestre pasado, para una Region.
- Product_Family_Total_Sells_Region_last_quarter_ratio: Product_Family_Total_Sells_Region_this_quarter / Product_Family_Total_Sells_Region_last_quarter

In [353]:
def time_feature_1(df):

    df['YearMonth'] = (df['Opportunity_Created_Date'].dt.year.astype(str) + df['Opportunity_Created_Date'].dt.month.astype(str)).astype(int)
    
    df = df.groupby(['Region','Product_Family']).apply(lambda x: x.sort_values('Opportunity_Created_Date'))
    
    df = df.reset_index(drop=True)
    
    df['Total_Product_Family_Region_This_Month'] = df.groupby(['Region','Product_Family', 'YearMonth'])['Opportunity_Created_Date'].transform('count')
    
    df['Total_Product_Family_Region_Last_Month'] = df.groupby(['Region', 'Product_Family'])['Total_Product_Family_Region_This_Month'].shift()
    
    df['Opp_Created_Month'] = df['Opportunity_Created_Date'].apply(lambda x: x.replace(day=1))
    
    df['Opp_Created_Date_last_row'] = df.groupby(['Region', 'Product_Family'])['Opp_Created_Month'].shift()
    
    df['Less_than_a_month'] = ((df['Opp_Created_Month'] - df['Opp_Created_Date_last_row']) / np.timedelta64(1, 'D')) <= 31
    
    df['Total_Product_Family_Region_Last_Month'] = df['Total_Product_Family_Region_Last_Month'] * df['Less_than_a_month']
    
    df['Total_Product_Family_Region_Last_Month'] = df['Total_Product_Family_Region_Last_Month'].fillna(0)
    
    df['Total_Product_Family_Region_Month_Change'] = df['Total_Product_Family_Region_Last_Month'] / df['Total_Product_Family_Region_This_Month']
    
    df = df.drop(columns=['YearMonth', 'Opp_Created_Month', 'Opp_Created_Date_last_row', 'Less_than_a_month'])
        
    return df

In [354]:
def time_feature_2(df):

    ## Total_Products_Region_This_Week, Total_Products_Region_Last_Week, Total_Products_Region_Week_Change
    
    df['YearWeek'] = (df['Opportunity_Created_Date'].dt.year.astype(str) + df['Opportunity_Created_Date'].dt.strftime('%V').astype(str)).astype(int)
    
    df = df.groupby(['Region']).apply(lambda x: x.sort_values('Opportunity_Created_Date'))
    
    df = df.reset_index(drop=True)
    
    df['Total_Products_Region_This_Week'] = df.groupby(['Region', 'YearWeek'])['Opportunity_Created_Date'].transform('count')
    
    df['Total_Products_Region_Last_Week'] = df.groupby(['Region'])['Total_Products_Region_This_Week'].shift()
    
    df['Opp_Created_Week'] = df['Opportunity_Created_Date'].apply(lambda x: x.replace(day=1))
    
    df['Opp_Created_Date_last_row'] = df.groupby(['Region'])['Opp_Created_Week'].shift()
    
    df['Less_than_a_week'] = ((df['Opp_Created_Week'] - df['Opp_Created_Date_last_row']) / np.timedelta64(1, 'D')) <= 7
    
    df['Total_Products_Region_Last_Week'] = df['Total_Products_Region_Last_Week'] * df['Less_than_a_week']
    
    df['Total_Products_Region_Last_Week'] = df['Total_Products_Region_Last_Week'].fillna(0)
    
    #Hago el ratio al reves para evitar infinitos
    
    df['Total_Products_Region_Week_Change'] = df['Total_Products_Region_Last_Week'] / df['Total_Products_Region_This_Week']
    
    df = df.drop(columns=['YearWeek', 'Opp_Created_Week', 'Opp_Created_Date_last_row', 'Less_than_a_week'])
    
    return df

In [355]:
def time_feature_3(df):

    ## Total_Products_Region_This_Month, Total_Products_Region_Last_Month, Total_Products_Region_Month_Change
    
    df['YearMonth'] = (df['Opportunity_Created_Date'].dt.year.astype(str) + df['Opportunity_Created_Date'].dt.month.astype(str)).astype(int)
    
    df = df.groupby(['Region']).apply(lambda x: x.sort_values('Opportunity_Created_Date'))
    
    df = df.reset_index(drop=True)
    
    df['Total_Products_Region_This_Month'] = df.groupby(['Region', 'YearMonth'])['Opportunity_Created_Date'].transform('count')
    
    df['Total_Products_Region_Last_Month'] = df.groupby(['Region'])['Total_Products_Region_This_Month'].shift()
    
    df['Opp_Created_Month'] = df['Opportunity_Created_Date'].apply(lambda x: x.replace(day=1))
    
    df['Opp_Created_Date_last_row'] = df.groupby(['Region'])['Opp_Created_Month'].shift()
    
    df['Less_than_a_month'] = ((df['Opp_Created_Month'] - df['Opp_Created_Date_last_row']) / np.timedelta64(1, 'D')) <= 31
    
    df['Total_Products_Region_Last_Month'] = df['Total_Products_Region_Last_Month'] * df['Less_than_a_month']
    
    df['Total_Products_Region_Last_Month'] = df['Total_Products_Region_Last_Month'].fillna(0)
    
    #Hago el ratio al reves para evitar infinitos
    
    df['Total_Products_Region_Month_Change'] = df['Total_Products_Region_Last_Month'] / df['Total_Products_Region_This_Month']
    
    df = df.drop(columns=['YearMonth', 'Opp_Created_Month', 'Opp_Created_Date_last_row', 'Less_than_a_month'])
    
    return df

In [356]:
def time_feature_4(df):

    ## Total_Products_Region_This_Quarter, Total_Products_Region_Last_Quarter, Total_Products_Region_Quarter_Change
    
    df['YearQuarter'] = (df['Opportunity_Created_Date'].dt.year.astype(str) + df['Opportunity_Created_Date'].dt.quarter.astype(str)).astype(int)
    
    df = df.groupby(['Region']).apply(lambda x: x.sort_values('Opportunity_Created_Date'))
    
    df = df.reset_index(drop=True)
    
    df['Total_Products_Region_This_Quarter'] = df.groupby(['Region', 'YearQuarter'])['Opportunity_Created_Date'].transform('count')
    
    df['Total_Products_Region_Last_Quarter'] = df.groupby(['Region'])['Total_Products_Region_This_Quarter'].shift()
    
    df['Opp_Created_Quarter'] = df['Opportunity_Created_Date'].apply(lambda x: x.replace(day=1))
    
    df['Opp_Created_Date_last_row'] = df.groupby(['Region'])['Opp_Created_Quarter'].shift()
    
    df['Less_than_a_Quarter'] = ((df['Opp_Created_Quarter'] - df['Opp_Created_Date_last_row']) / np.timedelta64(1, 'D')) <= 31*3
    
    df['Total_Products_Region_Last_Quarter'] = df['Total_Products_Region_Last_Quarter'] * df['Less_than_a_Quarter']
    
    df['Total_Products_Region_Last_Quarter'] = df['Total_Products_Region_Last_Quarter'].fillna(0)
    
    #Hago el ratio al reves para evitar infinitos
    
    df['Total_Products_Region_Quarter_Change'] = df['Total_Products_Region_Last_Quarter'] / df['Total_Products_Region_This_Quarter']
    
    df = df.drop(columns=['YearQuarter', 'Opp_Created_Quarter', 'Opp_Created_Date_last_row', 'Less_than_a_Quarter'])

    return df

In [357]:
def duration_features(df_train, df_test):
    
    ### Family_Duration
    
    df_familia = df_train[['Stage','Region','Product_Family','Planned_Delivery_Start_Date']]
    df_familia = df_familia[df_familia['Stage'] == 1]
    df_familia = df_familia.groupby(['Product_Family'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
    df_familia['Family_Duration'] = (df_familia['max'] - df_familia['min']).dt.days
    
    df_train = df_train.merge(df_familia[['Product_Family','Family_Duration','max']],on='Product_Family',how='left')
    df_train['Family_Duration'] =  (df_train['Planned_Delivery_Start_Date'] - df_train['max']).dt.days - df_train['Family_Duration']
    df_train['Family_Duration'].replace(np.nan,0)
    #df_train['Family_Duration'] =  np.sign(df_train['Family_Durarion'])
    df_train = df_train.drop(columns=['max'])
    df_test = df_test.merge(df_train[['Product_Family','Family_Duration']].drop_duplicates(subset=['Product_Family']),left_on='Product_Family',right_on='Product_Family',how='left')
    
    ### Region_Duration
    
    df_region = df_train[['Stage','Region','Planned_Delivery_Start_Date']]
    df_region = df_region[df_region['Stage'] == 1]
    df_region = df_region.groupby(['Region'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
    df_region['Region_Duration'] = (df_region['max'] - df_region['min']).dt.days
    df_train = df_train.merge(df_region[['Region','Region_Duration','max']],on='Region',how='left')
    df_train['Region_Duration'] =  (df_train['Planned_Delivery_Start_Date'] - df_train['max']).dt.days - df_train['Region_Duration']
    df_train['Region_Duration'].replace(np.nan,0)
    #df_train['Region_Duration'] =  np.sign(df_train['Region_Duration'])
    df_train = df_train.drop(columns=['max'])
    df_test = df_test.merge(df_train[['Region','Region_Duration']].drop_duplicates(subset=['Region']),left_on='Region',right_on='Region',how='left')
    
    ### Territory_Duration
    
    df_territory = df_train[['Stage','Territory','Planned_Delivery_Start_Date']]
    df_territory = df_territory[df_territory['Stage'] == 1]
    df_territory = df_territory.groupby(['Territory'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
    df_territory['Territory_Duration'] = (df_territory['max'] - df_territory['min']).dt.days
    df_train = df_train.merge(df_territory[['Territory','Territory_Duration','max']],on='Territory',how='left')
    df_train['Territory_Duration'] =  (df_train['Planned_Delivery_Start_Date'] - df_train['max']).dt.days - df_train['Territory_Duration']
    df_train['Territory_Duration'].replace(np.nan,0)
    #df_train['Territory_Duration'] =  np.sign(df_train['Territory_Duration'])
    df_train = df_train.drop(columns=['max'])
    df_test = df_test.merge(df_train[['Territory','Territory_Duration']].drop_duplicates(subset=['Territory']),left_on='Territory',right_on='Territory',how='left')
    
    return df_train, df_test

In [358]:
# Una idea de cuantas categorias hay por cada columna categorica.

columnas = df_train.select_dtypes(['category'])
resultado = []

for col in columnas:
    cuenta = df_train[col].value_counts().count()
    resultado.append((col, cuenta))
resultado.sort(key=lambda x: x[1])

for item in resultado:
    print(f"{item[0]} - {item[1]} categorias")

Delivery_Quarter - 4 categorias
Region - 5 categorias
ASP_Currency - 5 categorias
Total_Amount_Currency - 5 categorias
Total_Taxable_Amount_Currency - 5 categorias
Account_Type - 6 categorias
Bureaucratic_Code - 7 categorias
Delivery_Terms - 9 categorias
Opportunity_Type - 25 categorias
Account_Owner - 48 categorias
Opportunity_Owner - 53 categorias
Last_Modified_By - 53 categorias
Territory - 74 categorias
Billing_Country - 78 categorias
Product_Family - 225 categorias
Product_Name - 446 categorias
Account_Name - 1618 categorias
Sales_Contract_No - 5187 categorias
Opportunity_Name - 9700 categorias


In [359]:
def time_features(df):
    
    df = time_feature_1(df)
    df = time_feature_2(df)
    df = time_feature_3(df)
    df = time_feature_4(df)
    
    return df

def prepare_dataframes(df, dtypes_dict=None, test=False):
    
    if (dtypes_dict is not None):
        df = df.astype(dtypes_dict)
    
    df = df.sort_values('Opportunity_Created_Date').reset_index(drop=True)
    
    df = df.drop(columns=[#'ASP',
                          'Quote_Expiry_Date'
                         ])
    
    if (not test):
        df = df.drop(columns=['Opportunity_ID'])
    
    col = df.select_dtypes(['int64', 'float64']).columns
    
    
    df[col] = df[col].fillna(0)
    
    return df

# Componemos los dataframes para la red neuronal

#df_train = feature_creation(df_train.copy())
#df_train = feature_composition(df_train.copy())
#
#df_validation = feature_creation(df_validation.copy())
#df_validation = feature_composition(df_validation.copy())
#
#df_test = feature_creation(df_test.copy())
#df_test = feature_composition(df_test.copy())

#Combinamos train y validation para los features dependientes del tiempo
split_date_1 = df_validation['Opportunity_Created_Date'].nsmallest(1).values[0]
df_train_validation = df_train.append(df_validation, ignore_index=True)
split_date_2 = df_test['Opportunity_Created_Date'].nsmallest(1).values[0]
df_train_val_test = df_train_validation.append(df_test, ignore_index=True)
df_train_val_test = df_train_val_test.drop(columns=['Stage'])

df_train = time_features(df_train.copy())
df_train_validation = time_features(df_train_validation.copy())
df_train_val_test = time_features(df_train_val_test.copy())


########## VEMOS QUE PASA SI TOMAMOS LOS DATASETS COMPLETOS PARA CALCULAR LOS FEATURES

df_train = feature_creation(df_train.copy())
df_train = feature_composition(df_train.copy())

df_train_validation = feature_creation(df_train_validation.copy())
df_train_validation = feature_composition(df_train_validation.copy())

df_train_val_test = feature_creation(df_train_val_test.copy())
df_train_val_test = feature_composition(df_train_val_test.copy())

###################################################################################


#Volvemos a separar los dataframes

df_validation = df_train_validation[df_train_validation.Opportunity_Created_Date >= split_date_1]
df_test = df_train_val_test[df_train_val_test.Opportunity_Created_Date >= split_date_2]

df_train, df_validation = duration_features(df_train.copy(), df_validation.copy())
df_train_validation, df_test = duration_features(df_train_validation.copy(), df_test.copy())

# Recuperamos el tipado del set de test (que no entiendo por que se pierde)

dtypes_dict = df_train.drop(columns=['Stage']).dtypes.apply(lambda x: x.name).to_dict()

#Preparamos los df

df_train = prepare_dataframes(df_train, dtypes_dict=dtypes_dict)
df_validation = prepare_dataframes(df_validation, dtypes_dict=dtypes_dict, test=True)
df_test = prepare_dataframes(df_test, dtypes_dict=dtypes_dict, test=True)

In [360]:
a = len(df_train.columns)
b = len(df_validation.columns)
c = len(df_test.columns)

sub_b = df_train.columns.difference(df_validation.columns)
sub_c = df_train.columns.difference(df_test.columns)

print(a, b, c)
print(sub_b, sub_c)

88 89 88
Index([], dtype='object') Index(['Stage'], dtype='object')


In [361]:
df_train_val_test.dtypes

Region                                         category
Territory                                        object
Pricing, Delivery_Terms_Quote_Appr                int64
Pricing, Delivery_Terms_Approved                  int64
Bureaucratic_Code_0_Approval                      int64
                                                 ...   
Opportunity_Duration_by_Product_Family_mean     float64
Opportunity_Duration_by_Product_Family_std      float64
Total_Amount_by_Product_Family_mean             float64
Total_Amount_by_Product_Family_std              float64
Buro_Approved_by_Product_Family                 float64
Length: 86, dtype: object

In [362]:
ut.diagnostico_df(df_train)

Suma: 5292, Columna: Sales_Contract_No


In [363]:
ut.diagnostico_df(df_validation)

Suma: 1605, Columna: Sales_Contract_No


In [364]:
ut.diagnostico_df(df_test)

Suma: 181, Columna: Territory
Suma: 1213, Columna: Sales_Contract_No


## Feature Selection
#### Se almacenan todos los features creados para realizar feature selection en otro modulo

In [365]:
df_train.to_pickle("Archivos/Feature_Selection_entrenamiento.pkl")
df_validation.to_pickle("Archivos/Feature_Selection_validacion.pkl")

## Árboles: Se almacenan archivos

In [366]:
df_train.to_pickle("Archivos/Arboles_entrenamiento.pkl")
df_validation.to_pickle("Archivos/Arboles_validacion.pkl")
df_test.to_pickle("Archivos/Arboles_test.pkl")

In [367]:
if(not PREPARAR_REDES_NEURONALES): sys.exit()

## Redes Neuronales: Creamos el set de entrenamiento, validacion y test

In [368]:
def prepare_neural_network(df):
    #No poner Opportunity_ID aca
    #df = df.drop(columns=[#'Opportunity_Name',
    #                      #'Account_Name',
    #                      #'Account_Owner',
    #                      #'Opportunity_Owner',
    #                      #'Opportunity_Created_Date',
    #                      #'Account_Created_Date',
    #                      #'Last_Modified_Date',
    #                      #'Last_Modified_By',
    #                      #'Planned_Delivery_Start_Date',
    #                      #'Planned_Delivery_End_Date',
    #                      #'Month',
    #                      #'Product_Name',
    #                      #'Quote_Type',
    #                      #'Account_Type',
    #                      #'Account_Name',
    #                      #'ASP',
    #                      'Quote_Expiry_Date'
    #                     ])
    
    #df = df.select_dtypes(['category', 'int64', 'float64', 'datetime'])
    
    #Seleccion val_loss = 0.3858 -> Top 20 mejores features segun 'Feature Selection'
    #selected_columns = ['Sales_Contract_No', 'Opportunity_Name', 'Planned_Opportunity_Duration',
    #                    'Account_Name', 'Product_Name', 'Planned_Time_Until_Deliver',
    #                    'Bureaucratic_Code', 'Last_Modified_By', 'Opportunity_Owner',
    #                    'Product_Family', 'Opportunity_Type', 'Total_Amount(USD)',
    #                    'Account_Owner', 'Account_Type', 'Opportunity_Created_Date',
    #                    'Territory', 'Product_Amount_Deviation_of_Product_Family_rate',
    #                    'Planned_Deliver_Duration', 'Account_Created_Date', 'Billing_Country']
    
    #Seleccion de los mejores 15 features.
    
    
    selected_columns = ['Sales_Contract_No', 'Opportunity_Name', 'Planned_Opportunity_Duration',
                        'Account_Name', 'Product_Name', 'Planned_Time_Until_Deliver',
                        'Last_Modified_By', 'Bureaucratic_Code', 'Opportunity_Owner',
                        'Product_Family', 'Total_Amount(USD)', 'Opportunity_Type', 'Territory',
                        'Account_Owner', 'Account_Type', 'Opportunity_Created_Date',
                        'Planned_Deliver_Duration', 'Billing_Country',
                        'Opportunity_TRF_Region_std_Ratio', 'Total_Taxable_Amount_Currency']
    
    if 'Stage' in df.columns:
        selected_columns.append('Stage')
    
    if 'Opportunity_ID' in df.columns:
        selected_columns.append('Opportunity_ID')
    
    df = df[selected_columns]
    
    
    return df


df_train = prepare_neural_network(df_train.copy())
df_validation = prepare_neural_network(df_validation.copy())
df_test = prepare_neural_network(df_test.copy())

#### Buscamos salvar o eliminar datos incompatibles de los features recien creados
##### Nota: A partir de aca el trabajo es bastante manual, ya que depende de los features que creemos

In [369]:
# Dataframe de train
ut.diagnostico_df(df_train)

Suma: 5292, Columna: Sales_Contract_No


In [370]:
#df_train[df_train['ASP_by_Billing_Country_std'].isnull()]

In [371]:
#df_train['Opportunity_Taxable_Rate'] = df_train['Opportunity_Taxable_Rate'].fillna(0)
#ut.diagnostico_df(df_train)

In [372]:
# Dataframe de validacion
ut.diagnostico_df(df_validation)

Suma: 1605, Columna: Sales_Contract_No


In [373]:
#df_validation['Opportunity_Duration_Ratio'] = df_validation['Opportunity_Duration_Ratio'].fillna(0)
#ut.diagnostico_df(df_validation)

In [374]:
# Dataframe de test
#ut.diagnostico_df(df_test)

In [375]:
#df_test['Opportunity_Duration_Ratio'] = df_test['Opportunity_Duration_Ratio'].fillna(0)
#ut.diagnostico_df(df_test)

In [376]:
df_train.to_pickle("Archivos/Neuronales_entrenamiento.pkl")
df_validation.to_pickle("Archivos/Neuronales_validacion.pkl")
df_test.to_pickle("Archivos/Neuronales_test.pkl")

In [377]:
#prueba = df.loc[:, ['Region', 'Opportunity_Created_Date', 'Bureaucratic_Code_0_Approved']]
#prueba['This_Month_Sells'] = 1
#prueba['Last_Month_Sells'] = 1

#prueba['Last_Month_Sells'] = prueba.groupby([prueba.Region, prueba.Opportunity_Created_Date.dt.month])['count'].transform('count')
#this_month = prueba.groupby(prueba.Region).rolling('31d', on='Opportunity_Created_Date').agg({'Bureaucratic_Code_0_Approved' : 'count'}).reset_index()
#last_month = prueba.groupby(prueba.Region).rolling('62d', on='Opportunity_Created_Date').agg({'Bureaucratic_Code_0_Approved' : 'count'}).reset_index()
#this_month = this_month.drop(columns=['level_1']).rename(columns={'Bureaucratic_Code_0_Approved' : 'This_Month_Sells'}).set_index('Region')
#last_month = last_month.drop(columns=['level_1']).rename(columns={'Bureaucratic_Code_0_Approved' : 'Last_Month_Sells'}).set_index('Region')

#this_month